In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import norm

In [ ]:
# Supongamos que tenemos dos modelos, uno con 3 parámetros y otro con 5 parámetros.
# Definimos las funciones model1_run y model2_run

def model1_run(d, a, rho, sigmaConf, theta):
    # Aquí iría tu lógica específica para model1 con 3 parámetros
    pass

def model2_run(d, a, rho, sigmaConf, theta, alpha, beta):
    # Aquí iría tu lógica específica para model2 con 5 parámetros
    pass

# Función negativa de log-likelihood para model1 (con 3 parámetros)
def negative_log_likelihood_model1(params, data):
    rho, sigmaConf, theta = params
    all_secondOrder_mean_cor = []

    for n_p in data['Subj_idx'].unique():
        df_vp_participant = data[data['Subj_idx'] == n_p]
        d = df_vp_participant['Stimulus_transformed'].values
        a = df_vp_participant['Response_transformed'].values
        secondOrder_mean_cor = model1_run(d, a, rho, sigmaConf, theta)
        all_secondOrder_mean_cor.extend(secondOrder_mean_cor)

    all_secondOrder_mean_cor = np.array(all_secondOrder_mean_cor)
    conf_data = data.groupby('Subj_idx', group_keys=False).apply(lambda x: x.iloc[1:]).reset_index(drop=True)['Confidence_0to1'].values

    epsilon = 1e-10
    all_secondOrder_mean_cor = np.clip(all_secondOrder_mean_cor, epsilon, 1 - epsilon)
    log_likelihood = np.sum(norm.logpdf(conf_data, all_secondOrder_mean_cor))

    return -log_likelihood

# Función negativa de log-likelihood para model2 (con 5 parámetros)
def negative_log_likelihood_model2(params, data):
    rho, sigmaConf, theta, alpha, beta = params
    all_secondOrder_mean_cor = []

    for n_p in data['Subj_idx'].unique():
        df_vp_participant = data[data['Subj_idx'] == n_p]
        d = df_vp_participant['Stimulus_transformed'].values
        a = df_vp_participant['Response_transformed'].values
        secondOrder_mean_cor = model2_run(d, a, rho, sigmaConf, theta, alpha, beta)
        all_secondOrder_mean_cor.extend(secondOrder_mean_cor)

    all_secondOrder_mean_cor = np.array(all_secondOrder_mean_cor)
    conf_data = data.groupby('Subj_idx', group_keys=False).apply(lambda x: x.iloc[1:]).reset_index(drop=True)['Confidence_0to1'].values

    epsilon = 1e-10
    all_secondOrder_mean_cor = np.clip(all_secondOrder_mean_cor, epsilon, 1 - epsilon)
    log_likelihood = np.sum(norm.logpdf(conf_data, all_secondOrder_mean_cor))

    return -log_likelihood

# Definimos una función para calcular AIC y BIC
def calculate_aic_bic(log_likelihood, num_params, num_observations):
    aic = 2 * num_params - 2 * log_likelihood
    bic = np.log(num_observations) * num_params - 2 * log_likelihood
    return aic, bic

# Cargar datos
df = pd.read_csv('data_Mazancieux_2018.csv')
suj = [1,2,3]
df_vp = df[(df['Task'] == 'VP') & (df['Subj_idx'].isin(suj))].copy()
df_vp.loc[:, 'Stimulus_transformed'] = df_vp['Stimulus'].replace({1: -1, 2: 1})
df_vp.loc[:, 'Response_transformed'] = df_vp['Response'].replace({1: 0, 2: 1})
df_vp['Confidence_0to1'] = df_vp['Confidence'] / 10

# Número de observaciones
num_observations = len(df_vp)

# Optimización de los parámetros para model1
initial_guess_model1 = [0.2, 0.5, 0.8]  # Supongamos que estos son los parámetros iniciales para model1
bounds_model1 = [(0, 1), (0.1, 2), (0.1, 0.9)]
result_model1 = minimize(negative_log_likelihood_model1, initial_guess_model1, args=(df_vp,), bounds=bounds_model1, method='Powell')
params_model1 = result_model1.x
log_likelihood_model1 = -result_model1.fun

# Optimización de los parámetros para model2
initial_guess_model2 = [0.2, 0.5, 0.8, 0.4, 0.6]  # Supongamos que estos son los parámetros iniciales para model2
bounds_model2 = [(0, 1), (0.1, 2), (0.1, 0.9), (0, 1), (0, 1)]
result_model2 = minimize(negative_log_likelihood_model2, initial_guess_model2, args=(df_vp,), bounds=bounds_model2, method='Powell')
params_model2 = result_model2.x
log_likelihood_model2 = -result_model2.fun

# Calcular AIC y BIC para cada modelo
num_params_model1 = len(params_model1)
num_params_model2 = len(params_model2)

aic_model1, bic_model1 = calculate_aic_bic(log_likelihood_model1, num_params_model1, num_observations)
aic_model2, bic_model2 = calculate_aic_bic(log_likelihood_model2, num_params_model2, num_observations)

# Comparar los resultados
results = [
    {'model': 'model1', 'log_likelihood': log_likelihood_model1, 'aic': aic_model1, 'bic': bic_model1},
    {'model': 'model2', 'log_likelihood': log_likelihood_model2, 'aic': aic_model2, 'bic': bic_model2}
]

# Convertir resultados a DataFrame para mejor visualización
results_df = pd.DataFrame(results)
print(results_df)